# **Rejected Response Inference**

## **Config**

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"  # sesuaikan dengan GPU yang tersedia

seed = 42
DATASET_PATH    = "../datasets2070/cqa_rl_prompt_completion.jsonl"
CACHE_FOLDER    = "../model_cache"
SFT_MODEL_NAMES = {
    "Meta-Llama-3.1-8B": os.path.join("../sft/sft_output_Meta-Llama-3.1-8B", "final_checkpoint"),
    "Aya-23-8B":     os.path.join("../sft/sft_output_Aya-23-8B", "final_checkpoint"), 
    "SeaLLMs-v3-7B": os.path.join("../sft/sft_output_SeaLLMs-v3-7B", "final_checkpoint"),
    "SEA-LION-v3-8B":    os.path.join("../sft/sft_output_SEA-LION-v3-8B", "final_checkpoint"),
    "Sahabat-AI-8B":     os.path.join("../sft/sft_output_Sahabat-AI-8B", "final_checkpoint"),
}
BATCH_SIZE      = 32

## **Import Libraries**

In [2]:
import gc
import time
import json
import random
import torch
import pandas as pd
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    set_seed
)

pd.set_option("display.max_colwidth", None)

/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **Utility Functions**

In [3]:
def set_global_seed(s: int = seed):
    random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    set_seed(s)

def load_sft_model_and_tokenizer(model_path: str, tokenizer_path: str, hf_token: str = None):
    set_global_seed()
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_path,
        use_auth_token=hf_token,
        local_files_only=True
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=quant_config,
        low_cpu_mem_usage=True,
        use_auth_token=hf_token,
        local_files_only=True
    )
    model.config.use_cache = False
    model.eval()
    return tokenizer, model

def generate_sft_batch(tokenizer, model, prompts, max_new_tokens=256):
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1792
    ).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [
        text[len(prompt):].split("\n", 1)[0]
        for text, prompt in zip(decoded, prompts)
    ]


In [4]:
set_global_seed()

## **Load Dataset**

In [5]:
df = pd.read_json(DATASET_PATH, lines=True)
print(f"Total examples: {len(df)}")
df.sample(3)

Total examples: 2372


,prompt,completion
834,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 25 (1) Keputusan RUPS diambil berdasarkan musyawarah untuk mufakat. (2) Dalam hal keputusan berdasarkan musyawarah untuk mufakat sebagaimana dimaksud pada ayat (1) tidak tercapai, keputusan diambil melalui pemungutan suara. (3) Pengambilan keputusan melalui pemungutan suara sebagaimana dimaksud pada ayat (2) wajib dilakukan dengan memperhatikan ketentuan kuorum kehadiran dan kuorum keputusan RUPS.\n\nQuestion: Bagaimana cara pengambilan keputusan RUPS menurut Pasal 25?\nAnswer:",Objek yang diatur meliputi musyawarah untuk mufakat dan pemungutan suara jika mufakat tidak tercapai.
1509,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 3\n(1) \nKebijakan sebagaimana dimaksud dalam Pasal 2 \nmeliputi: \na. \npenetapan tata cara pelaksanaan kegiatan di Pasar \nModal; \nb. \npenetapan tata cara dan batas waktu penyampaian \nlaporan berkala dan insidentil pelaku industri di \nPasar Modal; \nc. \npenetapan tata cara pemberian izin, persetujuan, \ndan/atau pendaftaran di bidang Pasar Modal; \nd. \npenetapan \njangka \nwaktu \nberlakunya \nizin, \npersetujuan, \npendaftaran, \ndan \npenggunaan \ndokumen di bidang Pasar Modal; \ne. \npelaksanaan penilaian kemampuan dan kepatutan \ncalon pihak utama pelaku industri di Pasar Modal; \nf. \npemberian perintah kepada Bursa Efek, Lembaga \nKliring \ndan \nPenjaminan \ndan/atau \nLembaga \nPenyimpanan dan Penyelesaian untuk menetapkan\nperaturan dan/atau kebijakan yang mendukung \nterwujudnya stabilitas Pasar Modal; dan/atau \ng. \npenetapan kebijakan lainnya. \n(2) \nPenerapan kebijakan sebagaimana dimaksud pada ayat \n(1) \ndilaksanakan \ndengan \ntetap \nmemperhatikan \npenerapan \nprinsip \nketerbukaan, \nkehati-hatian, \nmanajemen risiko, tata kelola perusahaan yang baik \ndan sesuai dengan kondisi Pasar Modal terkini. \n(3) \nDalam \nrangka \npengambilan \nkebijakan \ndan \nmengevaluasi kebijakan yang telah ditetapkan untuk \nmengantisipasi dampak akibat COVID-19, Otoritas Jasa \nKeuangan berwenang meminta data dan informasi \ntambahan kepada pelaku industri di bidang Pasar \nModal di luar kewajiban pelaporan sebagaimana \ndimaksud dalam ketentuan peraturan perundang-\nundangan di bidang Pasar Modal.\n\nQuestion: Apa dasar hukum utama untuk Pasal 3 ayat (1)?\nAnswer:",Saya tidak tahu terkait dasar hukum utama untuk Pasal 3 ayat (1).
100,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 32 \n\n(1) Lembaga Penjaminan yang tidak memenuhi ketentuan Pasal 3 ayat (3), Pasal 3 ayat (6), Pasal 6 ayat (2), Pasal 7, Pasal 8, Pasal 9, Pasal 10 ayat (1), Pasal 11, Pasal 19, Pasal 21 ayat (1), Pasal 21 ayat (4), Pasal 23 ayat (3), Pasal 23 ayat (5), Pasal 24, \n\n Pasal 25…\n\n-24- \n\nPasal 25, dan Pasal 26 Peraturan Otoritas Jasa Keuangan ini, dikenakan sanksi administratif berupa: a. surat peringatan; b. pembekuan kegiatan usaha; atau c. pencabutan izin usaha. \n\n\n(2) Sanksi administratif berupa surat peringatan sebagaimana dimaksud pada ayat (1) huruf a, diberikan secara tertulis paling banyak 3 (tiga) kali berturut-turut dengan masa berlaku masing-masing 60 (enam puluh) hari. \n\n\n(3) Dalam hal sebelum berakhirnya masa berlaku surat peringatan sebagaimana dimaksud pada ayat (2), Lembaga Penjaminan telah memenuhi ketentuan sebagaimana dimaksud pada ayat (1), Otoritas Jasa Keuangan mencabut sanksi peringatan. \n\n\n(4) Dalam hal masa berlaku sanksi surat peringatan ketiga sebagaimana dimaksud pada ayat (2) berakhir serta Lembaga Penjaminan tetap tidak memenuhi ketentuan sebagaimana dimaksud pada ayat (1), Oto

## **Generate Rejected Responses & Build Preference Pairs**

In [ ]:
# kita pakai satu SFT model (atau bisa loop jika mau multi-model)
for model_key, sft_path in SFT_MODEL_NAMES.items():
    print(f"\n▶ Generating rejected responses with SFT {model_key}")
    tokenizer_dir = os.path.join(CACHE_FOLDER, model_key)
    tokenizer, model = load_sft_model_and_tokenizer(
        sft_path, tokenizer_dir, hf_token=os.getenv("HF_TOKEN")
    )

    output_path = f"preference_{model_key}.jsonl"

    # warm-up…
    _ = model.generate(
        **tokenizer("Warm up", return_tensors="pt").to(model.device),
        max_new_tokens=1,
        pad_token_id=tokenizer.eos_token_id
    )

    records = []
    start = time.time()
    for i in tqdm(range(0, len(df), BATCH_SIZE), desc=f"Inferring {model_key}"):
        batch   = df.iloc[i : i + BATCH_SIZE]
        prompts = batch["prompt"].tolist()
        preds   = generate_sft_batch(tokenizer, model, prompts)

        for prompt, chosen, rejected in zip(prompts, batch["completion"], preds):
            records.append({
                "prompt":   prompt,
                "chosen":   chosen,
                "rejected": rejected
            })

    print(f"→ Inference time: {time.time()-start:.1f}s, total pairs: {len(records)}")

    # simpan raw preference file per model
    with open(output_path, "w", encoding="utf-8") as f:
        for rec in records:
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    print(f"✔️ Saved raw preference dataset to `{output_path}`")

    # cleaning duplicates
    df_pref = pd.read_json(output_path, lines=True)
    mask_same = df_pref["chosen"].str.strip() == df_pref["rejected"].str.strip()
    df_clean = df_pref[~mask_same].reset_index(drop=True)
    clean_path = output_path.replace(".jsonl", "_clean.jsonl")
    df_clean.to_json(clean_path, orient="records", lines=True, force_ascii=False)
    print(f"✔️ Cleaned preference dataset saved to `{clean_path}`")

    # tampil sampel
    display(df_clean.sample(5).reset_index(drop=True))

    # selesai untuk model ini
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()


▶ Generating rejected responses with SFT Meta-Llama-3.1-8B
[2025-06-09 03:11:05,259] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
Inferring Meta-Llama-3.1-8B:   0%|          | 0/75 [00:00<?, ?it/s]/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Inferring Meta-Llama-3.1-8B:   3%|▎         | 2/75 [01:18<47:51, 39.34s/it]

In [ ]:
import signal

os.kill(os.getpid(), signal.SIGTERM)